In [ ]:
from pomegranate import *

#Rain node has no parents
rain = Node(DiscreteDistribution({
    "none": 0.7,
    "light": 0.2,
    "heavy":0.1
}), name="rain")

#Track maintainace node is conditional on rain
maintenance = Node(ConditionalProbabilityTable([
    ["none","yes",0.4],
    ["none","no",0.6],
    ["light","yes",0.2],
    ["light","no",0.8],
    ["heavy","yes",0.1],
    ["heavy","no",0.9]],
    [rain.distribution]), name="maintenance")

#Track node is conditional on rain and maintenance
train = Node(conditionalProbabilityTable([
    ["none","yes","on time",0.8],
    ["none","yes","delayed",0.2],
    ["none","no","on time",0.9],
    ["none","no","delayed",0.1],
    ["light","yes","on time",0.6],
    ["light","yes","delayed",0.4],
    ["light","no","on time",0.7],
    ["light","no","delayed",0.3],
    ["heavy","yes","on time",0.4],
    ["heavy","yes","delayed",0.6],
    ["heavy","no","on time",0.5],
    ["heavy","no","delayed",0.5],
],[rain.distribution,maintenance.distribution]), name="train")

#Appointment node is conditional on train
appointment = Node(ConditionalProbabilityTable([
    ["on time", "attend", 0.9],
    ["on time", "miss", 0.1],
    ["delayed", "attend",0.6],
], [train.distibution]), name="appointment")

#create a bayesian network and add states
model = BayesianNetwork()
model.add_states(rain,maintenance,train,appointment)

# Add edges connecting nodes
model.add_edge(rain, maintenance)
model.add_edge(rain, train)
model.add_edge(maintenance, train)

model.add_edge(train, appointment)

# Finalize model
model.bake()

In [ ]:
from pomegranate import *
try:
    # Define starting probabilities
    start = DiscreteDistribution({
        "sun":0.5,
        "rain":0.5
    })
    
    # Define transition model
    transistions = ConditionalProbabilityTable([
        ["sun", "sun", 0.8],
        ["sun", "rain",0.2],
        ["rain","sun",0.3],
        ["rain","rain",0.7]
    ], [start])
    
    # Create Markov chain
    model = MarkovChain([start, transistions])
    
    # sample 50 states from chain
    print(model.sample(50))
except Exception as e:
    print(e)

In [ ]:
pip install pomegranate

In [ ]:
pip install instructor

In [ ]:
pip install pydantic

In [ ]:
pip install openAi

In [ ]:
import instructor
from pydantic import BaseModel
from openai import OpenAI

# Define your desired output structure
class UserInfo(BaseModel):
    name:str
    age:int

#patch the openAi client
client = instructor.from_openai(OpenAI())

#Extract structured Data from natural language
user_info = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        response_model =UserInfo,
        message=[{"role":"user","content":"Brian Kimanzi is 22 years old."}],
)
print(user_info.name)
print(user_info.age)

In [ ]:
# A neural network that learns from a data and categorizes if the data is counterfeit or not
import csv
import tensorflow as tf

from sklearn.model_selection import train_test_split

# Read data in from file
with open('/Users/briankimanzi/Downloads/banknotes.csv') as f: #input file that contain the data
    reader = csv.reader(f)
    next(reader)

    data = []
    for row in reader:
        data.append({
            "evidence": [float(cell) for cell in row[:4]],
            "label": 1 if row[4] == "0" else 0
        })

# Separate data into training and testing groups
evidence = [row["evidence"] for row in data]
labels = [row["label"] for row in data]
x_training, x_testing, y_training, y_testing = train_test_split(
    evidence ,labels, test_size=0.4
    
)

# Create a neural network
model = tf.keras.models.Sequential()

# Add a hidden layer with 8 units, with ReLU activation
model.add(tf.keras.layers.Dense(8, input_shape=(4,), activation="relu"))

# Add output layer with 1 unit, with sigmoid activation
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

# Train neural network
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

model.fit(x_training, y_training, epochs=20)

# Evaluate how well model performs
model.evaluate(x_testing, y_testing, verbose=2)


In [ ]:
#filter Image
import math
import sys

from PIL import Image, ImageFilter

# Ensure correct usage
if len(sys.argv) !=2:
    sys.exit("Usage: Python filter.py filename")

# Open image
image = Image.open(sys.arvg[1]).convert("RGB") 

# Filter mage according to edge detection kernel
filtered = image.filter(ImageFilter.Kernel(
    size =(3, 3),
    kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
    scale=1
))
# show resulting image
filtered.show()

In [3]:
# Handwriting Recognision accuracy  
import sys
import tensorflow as tf

# use MNIST handwriting dataset
mnist = tf.keras.datasets.mnist # MNIST a dataset of a numerous handwritten samples of people handwritting 

# prepare data fro training
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
x_train = x_train.reshape(
    x_train.shape[0], x_train.shape[1], x_train.shape[2], 1
)
x_test = x_test.reshape(
    x_test.shape[0], x_test.shape[1], x_test.shape[2], 1
)
# Create a convolutional neural network
model = tf.keras.models.Sequential([

    # Convolutional layer, Learn 32 filters using a 3*3 kernel
    tf.keras.layers.Conv2D(
        32, (3, 3), activation="relu", input_shape=(28, 28, 1)
    ),

    # Max-Pooling layer, using 2*2 pool size
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Flatten units
    tf.keras.layers.Flatten(),

    # Add a hidden layer with dropout
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),

    # Add an output layer with output units for all 10 digits
    tf.keras.layers.Dense(10, activation="softmax") # softmax will take the output and turn it into a probabilty distribution
])

# Train neural network
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(x_train, y_train, epochs=10)

# Evaluate neural network performance
model.evaluate(x_test, y_test, verbose=2)

# Save model to file
if len(sys.argv) == 2:
    filename = sys.argv[1]
    model.save(filename)
    print(f"Model saved to {filename}.")
    

2024-06-24 10:49:06.609966: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/Cellar/jupyterlab/4.1.5/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 46s 23ms/step - accuracy: 0.8643 - loss: 0.4478
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.9652 - loss: 0.1129
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9762 - loss: 0.0785
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 59s 31ms/step - accuracy: 0.9808 - loss: 0.0653
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 52s 28ms/step - accuracy: 0.9819 - loss: 0.0577
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 88s 31ms/step - accuracy: 0.9854 - loss: 0.0445
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 64s 34ms/step - accuracy: 0.9871 - loss: 0.0385
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 34ms/step - accuracy: 0.9886 - loss: 0.0349
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 73s 29ms/step - accuracy: 0.9898 - loss: 0.0313
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 24ms/step - accuracy: 0.9898 - loss: 0.0308
313/313 - 2s - 6ms/step - accuracy: 0.9883 - loss: 0.0369


In [ ]:
import nltk

grammar = nltk.CFG.fromstring("""
# describe a context-free grammar
    s -> NP VP

    NP -> D N | N

    VP -> V | V NP

    D -> "the" | "a"
    N -> "she" | "city" | "car"
    V -> "saw" | "walked"
""")

parser = nltk.ChartParser(grammar)

sentence = input("sentence: ").split()
try:
    for tree in parser.parse(sentence):
        tree.pretty_print()

except ValueError:
    print("No parse tree possible.")

In [13]:
import nltk

grammar = nltk.CFG.fromstring("""
# describe a context-free grammar
    s -> NP VP

    AP -> A | A AP        
    NP -> N | D NP | AP NP | N PP       
    PP -> P NP          
    VP -> V | V NP | V NP PP        

    A -> "big" | "blue" | "small" | "dry" | "wide"
    D -> "the" | "a" | "an"
    N -> "she" | "city" | "car" | "street" | "dog" | "binoculars"
    P -> "on" | "over" | "before" | "below" | "with"
    V -> "saw" | "walked"
""")

parser = nltk.ChartParser(grammar)

sentence = input("sentence: ").split()
try:
    for tree in parser.parse(sentence):
        tree.pretty_print()

except ValueError:
    print("No parse tree possible.")

sentence:  she saw the dog with the binoculars


     s                                     
  ___|___________                           
 |               VP                        
 |    ___________|________                  
 |   |       |            PP               
 |   |       |        ____|___              
 |   |       NP      |        NP           
 |   |    ___|___    |     ___|______       
 NP  |   |       NP  |    |          NP    
 |   |   |       |   |    |          |      
 N   V   D       N   P    D          N     
 |   |   |       |   |    |          |      
she saw the     dog with the     binoculars

     s                                     
  ___|_______                               
 |           VP                            
 |    _______|___                           
 |   |           NP                        
 |   |    _______|____                      
 |   |   |            NP                   
 |   |   |    ________|___                  
 |   |   |   |            PP               
 |   |   |   |    ___

In [ ]:
import markovify
import sys

# Read text from file
if len(sys.argv) != 2:
    sys.exit("Usage python generator.py sample.txt")
with open(sys.argv[1]) as f:
    text = f.read()

# Train model
text_model = markovify.Text(text)

# Generate sentences 
print()
for i in range(5):
    print(text_model.make_sentence())
    print()